In [28]:
import importlib
import mysocket as sock

importlib.reload(sock)

<module 'mysocket' from '/Users/tcbressoud/Dropbox/cs181-DataSystems/cs181-bressoud/inclass_s20/source/03_19-second/mysocket.py'>

In [ ]:
webconn = sock.makeConnection("personal.denison.edu", 80)

In [ ]:
lines = ["GET / HTTP/1.1",
         "Host: personal.denison.edu",
         ""
        ]

In [ ]:
for line in lines:
    sock.sendString(webconn, line)
    sock.sendCRLF(webconn)

In [ ]:
s = sock.receiveTillClose(webconn)

In [ ]:
s

In [ ]:
webconn.close()

In [39]:
def getMailCode(serverline):
    endpos = serverline.index(" ")
    first = serverline[:endpos]
    num = int(first)
    return num

In [40]:
mailserver = sock.makeConnection("mail.denison.edu", 25)

In [41]:
s1 = sock.receiveToCRLF(mailserver)
status = getMailCode(s1)
print(repr(s1), status)

'220 smtpint2.cc.denison.edu ESMTP Postfix\r\n' 220


In [42]:
prefixlines = [
    "HELO 140.141.92.5",
    "MAIL FROM:<bressoud@denison.edu>",
    "RCPT TO:<tcbressoud@gmail.com>"
]

headers = [
    'From: "Dr. B" <bressoud@denison.edu>',
    'To: "Tom" <tcbressoud@gmail.com>',
    'Date: Sun, 29 Mar 2020 12:00:00 -0500',
    'Subject: Test message',
    ''
]

body = """Hello Tom-
This is a test message with four header lines and
five lines in the message body.

Best, Dr. B
"""

for line in prefixlines:
    sock.sendString(mailserver, line)
    sock.sendCRLF(mailserver)
    statusline = sock.receiveToCRLF(mailserver)
    assert getMailCode(statusline) == 250
    
sock.sendString(mailserver, "DATA")
sock.sendCRLF(mailserver)
statusline = sock.receiveToCRLF(mailserver)
assert getMailCode(statusline) == 354

for headerline in headers:
    sock.sendString(mailserver, headerline)
    sock.sendCRLF(mailserver)

for bodyline in body.split('\n'):
    sock.sendString(mailserver, bodyline)
    sock.sendCRLF(mailserver)

sock.sendString(mailserver, ".")
sock.sendCRLF(mailserver)

statusline = sock.receiveToCRLF(mailserver)
print(repr(statusline))
assert getMailCode(statusline) == 250

'250 2.0.0 Ok: queued as 26ACF409E344\r\n'


In [43]:
sock.sendString(mailserver, "QUIT")
sock.sendCRLF(mailserver)
statusline = sock.receiveToCRLF(mailserver)
assert getMailCode(statusline) == 221

In [44]:
mailserver.close()